# 행렬분할 

In [1]:
import json
import numpy as np
import tensorflow as tf
import time

In [2]:
with open("tag_similar_matrix30_1.json") as f:
    origin_mat = json.load(f)

In [3]:
origin_mat = np.array(origin_mat)

In [4]:
mat_A = origin_mat[:14580, :14580]
mat_B = origin_mat[:14580, 14580:]
mat_C = origin_mat[14580:, :14580]
mat_D = origin_mat[14580:, 14580:]

# 임시저장 (안해도됨)

In [5]:
mat_A = mat_A.tolist()
mat_B = mat_B.tolist()
mat_C = mat_C.tolist()
mat_D = mat_D.tolist()


In [7]:
with open("mat30_A.json", "w") as f:
    json.dump(mat_A,f)

In [18]:
with open("mat30_B.json", "w") as f:
    json.dump(mat_B,f)

In [9]:
with open("mat30_C.json", "w") as f:
    json.dump(mat_C,f)

In [10]:
with open("mat30_D.json", "w") as f:
    json.dump(mat_D,f)

# A부분

In [7]:
start = time.time()
tf.debugging.set_log_device_placement(True)
# 텐서를 GPU에 할당
with tf.device('/GPU:0'):
    A_tensor = tf.constant(mat_A, dtype = tf.float16)
    B_tensor = tf.constant(mat_B, dtype = tf.float16)
    C_tensor = tf.constant(mat_C, dtype = tf.float16)
    result_A1 = tf.matmul(A_tensor, A_tensor)
    result_A2 = tf.matmul(B_tensor, C_tensor)
    result_A = tf.add(result_A1, result_A2)
    
end = time.time()
print(end-start)
result_A = result_A.numpy().astype(np.int8)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
5.930923700332642


# B 부분

In [8]:
start = time.time()
tf.debugging.set_log_device_placement(True)
# 텐서를 GPU에 할당
with tf.device('/GPU:0'):
    A_tensor = tf.constant(mat_A, dtype = tf.float16)
    B_tensor = tf.constant(mat_B, dtype = tf.float16)
    D_tensor = tf.constant(mat_D, dtype = tf.float16)
    result_B1 = tf.matmul(A_tensor, B_tensor)
    result_B2 = tf.matmul(B_tensor, D_tensor)
    result_B = tf.add(result_B1, result_B2)
    
end = time.time()
print(end-start)
result_B = result_B.numpy().astype(np.int8)

2.0924341678619385


In [9]:
b1 = np.concatenate([result_A,result_B], axis = 1).tolist()

In [10]:
with open("tag_similar_matrix30_2_1.json", "w") as f:
    json.dump(b1, f)

# 커널 삭제 후 처음부터 시작해서 A B 실행 하지 말고 C D 만들기

# C 부분

In [5]:
start = time.time()
tf.debugging.set_log_device_placement(True)
# 텐서를 GPU에 할당
with tf.device('/GPU:0'):
    A_tensor = tf.constant(mat_A, dtype = tf.float16)
    C_tensor = tf.constant(mat_C, dtype = tf.float16)
    D_tensor = tf.constant(mat_D, dtype = tf.float16)
    result_C1 = tf.matmul(C_tensor, A_tensor)
    result_C2 = tf.matmul(D_tensor, C_tensor)
    result_C = tf.add(result_C1, result_C2)
    
end = time.time()
print(end-start)
result_C = result_C.numpy().astype(np.int8)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:0
4.579592227935791


# D 부분

In [6]:
start = time.time()
tf.debugging.set_log_device_placement(True)
# 텐서를 GPU에 할당
with tf.device('/GPU:0'):
    B_tensor = tf.constant(mat_B, dtype = tf.float16)
    C_tensor = tf.constant(mat_C, dtype = tf.float16)
    D_tensor = tf.constant(mat_D, dtype = tf.float16)
    result_D1 = tf.matmul(C_tensor, B_tensor)
    result_D2 = tf.matmul(D_tensor, D_tensor)
    result_D = tf.add(result_D1, result_D2)
    
end = time.time()
print(end-start)
result_D = result_D.numpy().astype(np.int8)

2.183192014694214


In [7]:
b2 = np.concatenate([result_C,result_D], axis =1).tolist()


In [8]:
with open("tag_similar_matrix30_2_2.json", "w") as f:
    json.dump(b2,f)

# 다시 커널 삭제후 b1, b2불러오기

In [ ]:
with open("tag_similar_matrix30_2_1.json", "r") as f:
    b1 = json.load(f)

In [ ]:
with open("tag_similar_matrix30_2_2.json", "r") as f:
    b2 = json.load(f)

In [ ]:
c = b1+b2

In [ ]:
with open("tag_similar_matrix30_2.json", "w") as f:
    json.dump(c,f)